# 계산 통계

### (1) 분포

통계에서 분포는 어떤 값과 그 값이 나타날 확률의 집합이다. <br />

파이썬으로 분포를 나타내고자 한다면 각 값과 확률을 연결하는 딕셔너리를 사용하면 된다. <br />
이런 파이썬 딕셔너리를 사용해서 여러 가지 유용한 기능을 제공하는 Pmf라는 클래스를 만들었다. <br />
이 클래스의 이름 Pmf는 분포를 수학적으로 표현하는 방법으로 확률 질량 함수(Probability mass function)에서 가져왔다.

Pmf는 이 책에서 사용하는 thinkbayes2에 정의된 파이썬 모듈이다. <br />
다음처럼 임포트해서 Pmf를 사용하면 된다.

In [2]:
from thinkbayes2 import Pmf

다음 코드는 Pmf로 육면체 주사위의 결과 분포를 나타낸다.

In [3]:
pmf = Pmf()
for x in [1,2,3,4,5,6]:
    pmf.Set(x, 1/6.0)

Pmf는 아무 값도 들어있지 않은 텅 빈 Pmf객체를 생성한다. Set메서드는 각 값에 1/6의 확률값을 설정한다.

다음 코드는 연속으로 나오는 각 단어가 등장하는 횟수를 세는 것이다.

In [ ]:
pmf = Pmf()
    for word in word_list:
    pmf.Incr(word, 1)

Incr은 단어마다 확률을 1씩 높인다. 단어가 Pmf에 없으면 단어를 추가한다. <br />
위 예제에서 확률은 아직 정규화 되지 않았다. 즉 전체 확률의 합이 1이 되지 않는다는 것이다. 이는 실제 확률이 아니다. <br />
하지만 이 예제에서 단어 횟수는 확률에 비례한다. 따라서 모든 단어를 세면 단어의 전체 수로 나누어 확률을 계산할 수 있다.

Pmf는 다음과 같이 사용하는 Normalize 메서드를 제공한다.

In [ ]:
pmf.Normalize()

Pmf 객체를 한번 만들면, 어느 값에 대해서도 확률을 물어볼 수 있다.

In [ ]:
print(pmf.Prob('the'))

Pmf는 파이썬 딕셔너리를 사용하여 값과 확률을 저장하므로 Pmf의 값은 어떤 해시 유형이든 될 수 있다. <br />
확률은  수치형이라면 어떤 식이든 상관없으나, 보통은 소수점을 사용하는 숫자(float형) 형태다.

### (2) 쿠키 문제

#### 전체 코드

In [7]:
from thinkbayes2 import Pmf

pmf = Pmf()
pmf.Set('Bowl 1', 0.5)
pmf.Set('Bowl 2', 0.5)

pmf.Mult('Bowl 1', 0.75)
pmf.Mult('Bowl 2', 0.5)

pmf.Normalize()

print(pmf.Prob('Bowl 1'))


0.6000000000000001


베이즈 이론을 살펴보면 당연히 Pmf를 사용하여 각 가설을 확률에 연결할 수 있다. <br />
쿠키 문제의 가설은 B(1)과 B(2)다. 파이썬으로 이를 문자열을 사용하여 나타내면 다음과 같다.

In [ ]:
pmf = Pmf()
pmf.Set('Bowl 1', 0.5)
pmf.Set('Bowl 2', 0.5)

각 가설에 대해서 사전 확률을 포함하는 사전 분포이다.

새로운 데이터(바닐라 쿠키)에 근거하여 이 분포를 갱신하려면 각 사전 확률과 이에 해당하는 우도를 곱한다. <br />
그릇 1에서 바닐라 쿠키를 집는 것에 대한 우도는 3/4이고, 그릇 2에서의 우도는 1/2이다

In [ ]:
pmf.Mult('Bowl 1', 0.75)
pmf.Mult('Bowl 2', 0.5)

Mult는 주어진 가설에 대한 확률을 찾아서 주어진 우도를 곱한다.

이렇게 갱신하면 이 가설들은 더 이상 정규화 되어있지 않다. <br /> 
하지만 상호 배반적이며 전체 포괄적이므로 이를 다시 정규화 해준다.

In [ ]:
pmf.Normalize()

결과는 각 가설에 대한 사후 확률을 포함하는 분포로 나타낸다. 이를 사후 분포라고 한다.

마지막으로 그릇 1의 사후 확률을 구할 수 있다.

In [ ]:
print (pmf.Prob('Bowl 1'))

### (3) 베이지안 프레임워크

#### 전체 코드

In [8]:
from thinkbayes2 import Pmf

class Cookie(Pmf):
    """A map from string bowl ID to probablity."""

    def __init__(self, hypos):
        """Initialize self.

        hypos: sequence of string bowl IDs
        """
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()

    def Update(self, data):
        """Updates the PMF with new data.

        data: string cookie type
        """
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        self.Normalize()

    mixes = {
        'Bowl 1':dict(vanilla=0.75, chocolate=0.25),
        'Bowl 2':dict(vanilla=0.5, chocolate=0.5),
        }

    def Likelihood(self, data, hypo):
        """The likelihood of the data under the hypothesis.

        data: string cookie type
        hypo: string bowl ID
        """
        mix = self.mixes[hypo]
        like = mix[data]
        return like


def main():
    hypos = ['Bowl 1', 'Bowl 2']

    pmf = Cookie(hypos)

    pmf.Update('vanilla')

    for hypo, prob in pmf.Items():
        print(hypo, prob)


if __name__ == '__main__':
    main()

Bowl 1 0.6000000000000001
Bowl 2 0.4


앞 장에서 다룬 코드를 좀 더 일반적인 형식으로 다시 작성하고자 한다. <br />
일단 이 문제에 관련된 코드를 캡슐화 해서 클래스로 정의하도록 하겠다.

> #### Cf) 캡슐화: 접근 제한자의 사용으로 프로그램 내부 보안을 강화시킨다.
>1. 객체에 포함된 정보의 손상과 오용을 막을 수 있다.
>2. 객체 내부의 조작 방법이 바뀌어도 사용 방법은 바뀌지 않는다.
>3. 데이터가 바뀌어도 다른 객체에 영향을 주지않아 독립성이 유지된다.
>4. 처리된 결과만 사용하므로 객체의 이식성이 좋다.
>5. 객체를 부품화 할 수 있어 새로운 시스템의 구성에 부품처럼 사용할 수 있다.

In [ ]:
class Cookie(Pmf):
    
    def __init__(self, hypos):
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()

Cookie는 가설에서 이에 대한 확률을 연결하는 Pmf객체다. <br />
__init__ 메서드는 각 가설 별로 동일한 사전 확률을 부여한다. <br />
1장에서 처럼 여기서도 두 개의 가설을 사용한다.

In [ ]:
    hypos = ['Bowl 1', 'Bowl 2']
    pmf = Cookie(hypos)

Cookie는 데이터를 매개 변수로 취해서 각각의 확률을 갱신하는 Update 메서드를 제공한다.

In [ ]:
    def Update(self, data):
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        self.Normalize()

Update는 스윗에서 각 가설을 반복 실행하여 각 가설하에 Likelihood를 통해 계산된 데이터의 우도를 확률에 곱한다.

In [ ]:
    mixes = {
        'Bowl 1':dict(vanilla=0.75, chocolate=0.25),
        'Bowl 2':dict(vanilla=0.5, chocolate=0.5),
        }
    def Likelihood(self, date, hypo):
        mix = self.mixes[hypo]
        like = mix[data]
        return like

Likelihood는 그릇의 이름을 그릇 안에 다양한 쿠키에 연결하는 딕셔너리 형태인 mixes를 사용한다.

다음처럼 갱신할 수 있다.

In [ ]:
    pmf.Update('vanilla')

그러면 각 가설에 대해서 사후 확률을 출력할 수 있다.

In [ ]:
    for hypo, prob in pmf.Items():
        print (hypo, prob)

이 코드는 1장에서 본 것보다 더 복잡하다. <br /> 
한 가지 장점은 같은 그릇에서, 반복해서 한 개 이상의 쿠키를 집는 경우를 일반화 했다는 것이다.

In [ ]:
    dataset = ['vanilla', 'chocolate', 'vanilla']
        for data in dataset:
            pmf.Update(data)

In [ ]:
또한 유사한 여러 문제를 풀 수 있는 프레임 워크를 제공한다는 장점도 있다. 

### (4) 몬티 홀 문제

#### 전체 코드

In [9]:
from thinkbayes2 import Pmf

class Monty(Pmf):
    """Map from string location of car to probability"""

    def __init__(self, hypos):
        """Initialize the distribution.

        hypos: sequence of hypotheses
        """
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()

    def Update(self, data):
        """Updates each hypothesis based on the data.

        data: any representation of the data
        """
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        self.Normalize()

    def Likelihood(self, data, hypo):
        """Compute the likelihood of the data under the hypothesis.

        hypo: string name of the door where the prize is
        data: string name of the door Monty opened
        """
        if hypo == data:
            return 0
        elif hypo == 'A':
            return 0.5
        else:
            return 1


def main():
    hypos = 'ABC'
    pmf = Monty(hypos)

    data = 'B'
    pmf.Update(data)

    for hypo, prob in sorted(pmf.Items()):
        print(hypo, prob)


if __name__ == '__main__':
    main()


A 0.3333333333333333
B 0.0
C 0.6666666666666666


몬티 홀 문제를 풀기 위해 새로운 클래스를 정의한다.

In [ ]:
class Monty(Pmf):
    def __init__(self, hypos):
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()

Monty와 Cookie는 완전히 동일하다. Pmf를 생성하는 코드도 가설 이름을 제외하면 동일하다.

In [ ]:
    hypos = 'ABC'
    pmf = Monty(hypos)

Update를 호출하는 방법도 꽤 유사하다.

In [ ]:
    data = 'B'
    pmf.Update(data)

Update를 구현하는 방법도 완전히 같다.

In [ ]:
    def Update(self, data):
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        self.Normalize()

하지만 Likelihood는 약간 수정해야 한다.

In [ ]:
    def Likelihood(self, data, hypo):
        if hypo == data:
            return 0
        elif hypo == 'A':
            return 0.5
        else
            return 1

끝으로 결과를 출력하는 부분도 같다.

In [ ]:
    for hypo, prob in pmf.Items():
        print(hypo, prob)

이 예제에서 Likelihood를 작성하는 부분이 조금 복잡하지만, 베이지안 확률 갱신 프레임워크는 단순하다.

### (5) 프레임워크 캡슐화 

#### 전체 코드

In [10]:
from thinkbayes2 import Suite


class Monty(Suite):
    def Likelihood(self, data, hypo):
        """Computes the likelihood of the data under the hypothesis.

        hypo: string name of the door where the prize is
        data: string name of the door Monty opened
        """
        if hypo == data:
            return 0
        elif hypo == 'A':
            return 0.5
        else:
            return 1


def main():
    suite = Monty('ABC')
    suite.Update('B')
    suite.Print()


if __name__ == '__main__':
    main()


A 0.3333333333333333
B 0.0
C 0.6666666666666666


프레임워크의 어떤 요소가 동일한지를 파악하였으니, 이제 이를 객체(__init__, Update, Print함수를 제공하는 Pmf라는 스윗)로 캡슐화할 수 있다.

In [ ]:
Class Suite(Pmf):
    """가설과 가설들의 확률로 구성된 스윗을 나타냄"""
    
    def __init__(self, hypo=tuple()):
        """분포 초기화"""
        
    def Update(self, data):
        """데이터 기반의 가설을 각각 갱신"""
    
    def Print(self):
        """가설과 확률 출력"""

Suite을 사용하려면 이를 상속하는 클래스를 작성하고 Likelihood를 추가해야 한다. <br />
다음은 Suite을 사용하여 몬티 홀 문제를 다시 작성한 것이다.

In [ ]:
class Monty(Suite):
    def Likelihood(self, data, hypo):
        """Computes the likelihood of the data under the hypothesis.

        hypo: string name of the door where the prize is
        data: string name of the door Monty opened
        """
        if hypo == data:
            return 0
        elif hypo == 'A':
            return 0.5
        else:
            return 1

다음은 이 클래스를 사용한 코드다.

In [ ]:
def main():
    suite = Monty('ABC')
    suite.Update('B')
    suite.Print()

### (6) M&M 문제

#### 전체 코드

In [11]:
from thinkbayes2 import Suite


class M_and_M(Suite):
    """Map from hypothesis (A or B) to probability."""

    mix94 = dict(brown=30,
                 yellow=20,
                 red=20,
                 green=10,
                 orange=10,
                 tan=10,
                 blue=0)

    mix96 = dict(blue=24,
                 green=20,
                 orange=16,
                 yellow=14,
                 red=13,
                 brown=13,
                 tan=0)

    hypoA = dict(bag1=mix94, bag2=mix96)
    hypoB = dict(bag1=mix96, bag2=mix94)

    hypotheses = dict(A=hypoA, B=hypoB)

    def Likelihood(self, data, hypo):
        """Computes the likelihood of the data under the hypothesis.

        hypo: string hypothesis (A or B)
        data: tuple of string bag, string color
        """
        bag, color = data
        mix = self.hypotheses[hypo][bag]
        like = mix[color]
        return like


def main():
    suite = M_and_M('AB')

    suite.Update(('bag1', 'yellow'))
    suite.Update(('bag2', 'green'))

    suite.Print()


if __name__ == '__main__':
    main()


A 0.7407407407407407
B 0.2592592592592592


M&M 문제를 푸는 데도 Suite을 사용할 수 있다. <br />
우선 해야 할 것은 1995년 이전과 이후의 초콜렛의 혼합된 색을 코드로 표현하는 것이다.

In [ ]:
    mix94 = dict(brown=30,
                 yellow=20,
                 red=20,
                 green=10,
                 orange=10,
                 tan=10,
                 blue=0)

    mix96 = dict(blue=24,
                 green=20,
                 orange=16,
                 yellow=14,
                 red=13,
                 brown=13,
                 tan=0)

그 다음으로는 가설을 코드화 해야한다.

In [ ]:
    hypoA = dict(bag1=mix94, bag2=mix96)
    hypoB = dict(bag1=mix96, bag2=mix94)

hypoA는 1번 봉지가 1994년에 나온 것이고 2번 봉지는 1996년에 나온 것이라는 가설을 나타낸다. <br />
hypoB는 반대 내용이다.

다음으로 가설의 이름을 이 내용에 연결시킨다. 

In [ ]:
    hypotheses = dict(A=hypoA, B=hypoB)

마지막으로 Likelihood를 작성할 수 있다. 가설 hypo의 경우 A든 B든 문자열로 되어있다. <br />
데이터는 봉지(bag)와 색을 나타내는 튜플로 되어 있다.

In [ ]:
def Likelihood(self, data, hypo):
        bag, color = data
        mix = self.hypotheses[hypo][bag]
        like = mix[color]
        return like

다음은 스윗을 생성하고 갱신하는 코드다

In [ ]:
    suite = M_and_M('AB')

    suite.Update(('bag1', 'yellow'))
    suite.Update(('bag2', 'green'))

    suite.Print()

### (7) 토의
이 장에서는 베이지안 갱신 프레임워크를 캡슐화하는 Suite 클래스를 설명했다. <br />
Suite은 Suite이 가지고 있을 것이라고 가정하지만 실제로 완전히 구현되지 않은 인터페이스를 정의하는 것을 의미하는 추상적 타입이다. <br />
Suite 인터페이스는 Update와 Likelihood를 포함하지만, Suite클래스는 Update에 대한 구현 내용만 제공할 뿐 <br />
Likelihood에 대한 내용은 없다. <br />
구체적 타입은 추상적 부모 클래스를 확장하여 빠진 메서드 구현을 추가한 클래스다. <br />
예를 들자면, Monty는 Suite을 확장해서 Update를 상속받고 Likelihood를 구현해서 제공한다. <br />
앞으로 각각의 문제에서 Suite을 확장하고, Update를 상속받고, Likelihood를 작성하는 새로운 클래스를 정의할 것이다. 
일부의 경우 보통 성능 향상을 위해 Update를 오버라이드 하기도 한다. 

#### Cf) 오버라이드: 메서드 오버라이드(override)는 자식 클래스에서 부모 클래스의 기능(method)를 재정의할 때 사용하는 기능 <br />
 주로 두가지 경우에서 사용
> - 1: 부모 클래스의 기능을 사용하지 않고 자식 클래스에서 구현한 기능을 사용하고 싶은 경우
> - 2: 부모 클래스의 기능을 자식 클래스에서 확장하고 싶은 경우


